### Import modules

In [1]:
from google import genai
from google.genai import types

### Function Declaration

In [2]:
get_temperature_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. 'San Francisco'.",
            },
        },
        "required": ["location"],
    },
}

### Function Implementation

In [3]:
def get_current_temperature(location: str) -> dict:
    """Dummy function that returns a fixed temperature."""
    print(f"Fetching temperature for {location}... (dummy function)")
    return {"temperature_celsius": 27}

### Initialize Client

In [4]:
client = genai.Client()
tools = types.Tool(function_declarations=[get_temperature_function])
config = types.GenerateContentConfig(tools=[tools])

### Send the User Message (Turn 1)

In [5]:
user_message = types.Content(role="user", parts=[types.Part(text="What's the temperature in London?")])

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[user_message],
    config=config,
)

### Extract, Invoke the Function Call and Pass Response (Turn 2 & 3)

In [ ]:
function_call = response.candidates[0].content.parts[0].function_call

if function_call:
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")

    # Call the backend function manually
    function_result = get_current_temperature(**function_call.args)
    print(f"Function result: {function_result}")

    # Build the function call content (as Gemini generated it)
    model_function_call = types.Content(
        role="model",
        parts=[types.Part(function_call=function_call)]
    )

    # Build the function response content
    function_response_content = types.Content(
        role="function",
        parts=[types.Part(function_response=types.FunctionResponse(
            name=function_call.name,
            response=function_result
        ))]
    )

    follow_up_response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            user_message,             # Turn 1: User
            model_function_call,      # Turn 2: Model's function call
            function_response_content # Turn 3: Function response
        ]
    )

    final_text = follow_up_response.candidates[0].content.parts[0].text
    print(f"\nGemini's final response: {final_text}")

else:
    print("No function call found in the response.")